# SeQR_BackEnd

In [ ]:
pip install BeautifulSoup

  Using cached BeautifulSoup-3.2.2.tar.gz (32 kB)
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [ ]:
import requests

def get_final_url(url):
    try:
        # HTTP HEAD 요청을 보내고 리디렉션을 추적
        response = requests.head(url, allow_redirects=True)
        return response.url  # 최종 URL 반환
    except requests.exceptions.RequestException as e:
        return {"error": str(e)}  # 요청 중 발생한 예외 처리

# 사용 예시
url = 'https://bIt.ly/2024먼동제부스신청'
final_url = get_final_url(url)
print(final_url)


https://docs.google.com/forms/d/e/1FAIpQLSchvOglWaMflxy7fc6GyvbJrKfDRGmLu_ORC-lD_AatH7Pb9A/closedform


In [ ]:
import requests
from bs4 import BeautifulSoup

def get_url_preview(url):
    try:
        response = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'})
        response.raise_for_status()  # 요청이 성공했는지 확인
        soup = BeautifulSoup(response.text, 'html.parser')

        # Open Graph 및 기본 메타 태그에서 정보 추출
        title = soup.find("meta", property="og:title") or soup.find("title")
        description = soup.find("meta", property="og:description") or soup.find("meta", attrs={"name": "description"})
        image = soup.find("meta", property="og:image")

        # 메타 태그가 없을 경우 대비
        preview_info = {
            "title": title["content"] if title and title.has_attr("content") else title.string if title else "No title found",
            "description": description["content"] if description else "No description found",
            "image": image["content"] if image else "No image found"
        }

        return preview_info

    except requests.exceptions.RequestException as e:
        return {"error": str(e)}

# 사용 예시
url = "https://ticket.interpark.com/webzine/paper/TPNoticeView.asp?bbsno=34&no=53330"
preview = get_url_preview(url)
print(preview)


{'title': '인터파크 티켓', 'description': 'No description found', 'image': 'No image found'}


In [ ]:
from fastapi import FastAPI
from pydantic import BaseModel
from seqr_model import predict_url  # seqr_model에서 predict_url 함수 불러오기
import requests
from bs4 import BeautifulSoup

# FastAPI 애플리케이션 초기화
app = FastAPI()

# 요청 바디를 정의하는 Pydantic 모델
class TextRequest(BaseModel):
    text: str

# 최종 URL을 가져오는 함수
def get_final_url(url):
    try:
        response = requests.head(url, allow_redirects=True)
        return response.url
    except requests.exceptions.RequestException as e:
        return {"error": str(e)}

# URL 미리보기 정보 가져오는 함수
def get_url_preview(url):
    try:
        response = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'})
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')

        title = soup.find("meta", property="og:title") or soup.find("title")
        description = soup.find("meta", property="og:description") or soup.find("meta", attrs={"name": "description"})
        image = soup.find("meta", property="og:image")

        preview_info = {
            "title": title["content"] if title and title.has_attr("content") else title.string if title else "No title found",
            "description": description["content"] if description else "No description found",
            "image": image["content"] if image else "No image found"
        }
        return preview_info
    except requests.exceptions.RequestException as e:
        return {"error": str(e)}

@app.post("/predict")
async def predict(request: TextRequest):
    # seqr_model.py의 predict_url 함수 사용하여 예측 수행
    predicted_class, probabilities = predict_url(request.text)

    # URL 처리
    urls = [word for word in request.text.split() if word.startswith("http")]
    url_data = []
    for url in urls:
        final_url = get_final_url(url)
        preview_info = get_url_preview(url)
        url_data.append({
            "url": url,
            "final_url": final_url,
            "preview": preview_info
        })

    return {
        "predicted_class": predicted_class,
        "probabilities": probabilities,
        "original_text": request.text,
        "url_data": url_data
    }


ModuleNotFoundError: No module named 'fastapi'